<a href="https://colab.research.google.com/github/EvgeniaKomleva/DeepPavlovBot/blob/master/EnSimpleBotWithDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deeppavlov
!python -m deeppavlov install gobot_dstc2_minimal

2020-07-30 10:07:06.78 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_dstc2_minimal' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_dstc2_minimal.json'


## 0. Подготовка данных

Тут нужно создать папку assistant_data и поместить в нее файлы (четыре)
После этого можно запускать дальше
Считываем данные:

In [ ]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader


class AssistantDatasetReader(SimpleDSTC2DatasetReader):
    
    
    @staticmethod
    def _data_fname(datatype):
        assert datatype in ('val', 'trn', 'tst'), "wrong datatype name"
        return f"assistant-{datatype}.json"

In [ ]:
data = AssistantDatasetReader().read('assistant_data')

2020-07-30 10:07:19.646 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from assistant_data/assistant-trn.json]
2020-07-30 10:07:19.648 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from assistant_data/assistant-val.json]
2020-07-30 10:07:19.651 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from assistant_data/assistant-tst.json]
2020-07-30 10:07:19.653 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 14 samples in train split.
2020-07-30 10:07:19.655 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 3 samples in valid split.
2020-07-30 10:07:19.656 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 2 samples in test split.


In [ ]:
!ls assistant_data

assistant-templates.txt  assistant-tst.json
assistant-trn.json	 assistant-val.json


In [ ]:
!head -n 310 assistant_data/assistant-trn.json

[
  [
    {
      "speaker": 1,
      "text": "Hi"
    },
    {
      "speaker": 2,
      "text": "Hello, whant credit??",
      "act": "Hello"
    },
    {
      "speaker": 1,
      "text": "No"
    },
    {
      "speaker": 2,
      "text": "Good bye!",
      "act": "good_bye"
    }
  ],
  [
    {
      "speaker": 1,
      "text": "Hello"
    },
    {
      "speaker": 2,
      "text": "Hello, whant credit??",
      "act": "Hello"
    },
    {
      "speaker": 1,
      "text": "Yes"
    },
    {
      "speaker": 2,
      "text": "Consumer or Commercial?",
      "act": "about_credit"
    },
    {
      "speaker": 1,
      "text": "Consumer"
    },
    {
      "speaker": 2,
      "text": "Something about it",
      "act": "good_bye"
    }
  ],
  [
    {
      "speaker": 1,
      "text": "Good morning!"
    },
    {
      "speaker": 2,
      "text": "Hello, whant credit??",
      "act": "Hello"
    },
    {
      "speaker": 1,
      "text": "Yes"
    },
    {
      "speaker": 2,
      "t

Делаем итератор по данным


In [ ]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed dialogs:

In [ ]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

print("\n-----------------")    
print(f"{len(iterator.get_instances('train')[0])} dialog(s) in train.")
print(f"{len(iterator.get_instances('valid')[0])} dialog(s) in valid.")
print(f"{len(iterator.get_instances('test')[0])} dialog(s) in test.")

User utterances:
----------------

[   {'prev_resp_act': None, 'text': 'Hello'},
    {'prev_resp_act': 'Hello', 'text': 'Yes'},
    {'prev_resp_act': 'about_credit', 'text': 'Consumer'}]

System responses:
-----------------

[   {'act': 'Hello', 'text': 'Hello, whant credit??'},
    {'act': 'about_credit', 'text': 'Consumer or Commercial?'},
    {'act': 'good_bye', 'text': 'Something about it'}]

-----------------
5 dialog(s) in train.
1 dialog(s) in valid.
1 dialog(s) in test.


## 1. Учим бота



&nbsp;
![gobot_simple_templates.png](https://github.com/deepmipt/DeepPavlov/blob/master/examples/img/gobot_simple_templates.png?raw=1)
&nbsp;

Шаблоны должны быть в формате `act` TAB `<template>` где `<act> `- это диалоговое действие, а `<template>` - соответствующий ответ.



Список действий для нашего бота

In [ ]:
!head -n 10 assistant_data/assistant-templates.txt

Hello	Maybe want credit?
suggest_credit	Maybe want credit?
suggest_money	Maybe want money?
suggest_cycling	Ask once more
good_bye	Bye!
bad_bye		Good lack!
about_credit	Which credit?


По сути, модуль диалога решает задачу классификации, где набор классов определяется в `assistant-templates.txt`. Таким образом, для обучения сети политики диалога вам понадобится ярлык действия для каждой очереди системы в обучающих диалогах. Наш набор данных помощника предоставляет `" act "` ключ словаря, который содержит действие, связанное с текущим ответом. Вот пример данных обучения для сети.

In [ ]:
!head -n 31 assistant_data/assistant-trn.json

[
  [
    {
      "speaker": 1,
      "text": "Hi"
    },
    {
      "speaker": 2,
      "text": "Hello, whant credit??",
      "act": "Hello"
    },
    {
      "speaker": 1,
      "text": "No"
    },
    {
      "speaker": 2,
      "text": "Good bye!",
      "act": "good_bye"
    }
  ],
  [
    {
      "speaker": 1,
      "text": "Hello"
    },
    {
      "speaker": 2,
      "text": "Hello, whant credit??",
      "act": "Hello"
    },


Навыки в DeepPavlov определяются конфигурационными файлами. Итак, мы будем использовать минимальную конфигурацию бота DSTC2 (доступно больше конфигураций) и поменять разделы, отвечающие за:
- embeddings,
- шаблоны ответов,
- пути загрузки и сохранения данных и моделей.

Загрузим бот:

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2_minimal)

Загружаем GLOVe embeddings (ВАЖНО! Эмббединги для англиского языка, т е бот обучается на английском словаре, нужно найти русские эмббединги):

In [ ]:
from deeppavlov.download import download_resource

download_resource(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
                  dest_paths=['assistant_bot/'])

2020-07-30 10:07:27.114 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes


Изменяем конфигурации


In [ ]:
gobot_config['chainer']['pipe'][-1]['embedder'] = {
    "class_name": "glove",
    "load_path": "assistant_bot/glove.6B.100d.txt"
}

In [ ]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'assistant_data/assistant-templates.txt'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['api_call_action'] = None

In [ ]:
gobot_config['dataset_reader']['class_name'] = '__main__:AssistantDatasetReader'
gobot_config['metadata']['variables']['DATA_PATH'] = 'assistant_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'assistant_bot'

In [ ]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 # set batch size
gobot_config['train']['max_batches'] = 30 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 30 # evaluate on full 'valid' split every 30 epochs
gobot_config['train']['log_every_n_batches'] = 5 # evaluate on full 'train' split every 5 batches

train_model(gobot_config);

2020-07-30 10:07:27.161 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/assistant_data/assistant-trn.json]
2020-07-30 10:07:27.164 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/assistant_data/assistant-val.json]
2020-07-30 10:07:27.167 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/assistant_data/assistant-tst.json]
2020-07-30 10:07:27.169 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 14 samples in train split.
2020-07-30 10:07:27.172 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 3 samples in valid split.
2020-07-30 10:07:27.173 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 2 samples in test split.
2020-07-30 10:07:27.177 INFO in 'deeppavlov.core.data.simple_vocab'['simple_

{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 2, "batches_seen": 5, "train_examples_seen": 14, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.5232914447784425}}
{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 4, "batches_seen": 10, "train_examples_seen": 25, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.7491014242172241}}
{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 7, "batches_seen": 15, "train_examples_seen": 39, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.26917733550071715}}


2020-07-30 10:08:14.678 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 211: Did not improve on the per_item_dialog_accuracy of 0.0


{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 9, "batches_seen": 20, "train_examples_seen": 50, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.08782302141189575}}
{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 12, "batches_seen": 25, "train_examples_seen": 64, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.04324427917599678}}
{"train": {"eval_examples_count": 5, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 14, "batches_seen": 30, "train_examples_seen": 75, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.03345228508114815}}
{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01", "epochs_done": 14, "batches_seen": 30, "train_examples_seen": 75, "impatience": 1, "patience_limit": 10}}


2020-07-30 10:08:14.797 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/assistant_bot/word.dict]
2020-07-30 10:08:14.800 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/assistant_bot/glove.6B.100d.txt`]
2020-07-30 10:09:00.789 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-30 10:09:00.952 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/assistant_bot/word.dict]
2020-07-30 10:09:00.957 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/assistant_bot/glove.6B.100d.txt`]


{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 0.0}, "time_spent": "0:00:01"}}


2020-07-30 10:09:47.123 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.




Можно поменять конфиг файл [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) 

# 2. Посмотрим как работает бот и запустим его в телеграмме 
Библиотека предоставляет возможности так же запускать на других платформах

In [ ]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2020-07-30 10:09:47.205 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/assistant_bot/word.dict]
2020-07-30 10:09:47.208 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/assistant_bot/glove.6B.100d.txt`]
2020-07-30 10:10:33.162 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.


In [ ]:
bot([[{"text": "Привет"}]])

[['Maybe want money?']]

In [ ]:
bot([[{"text": "как дела"}]])

[['Maybe want credit?']]

In [ ]:
bot([[{"text": "Нет"}]])

[['Bye!']]

In [ ]:
bot.reset()

In [ ]:
bot([[{"text": "Hello"}]])

[['Good lack!']]

In [ ]:
bot([[{"text": "No"}]])

[['Maybe want credit?']]

In [ ]:
bot([[{"text": "Consumer"}]])

[['Maybe want money?']]

Посмотрим как это выглядит в телеграмме

In [ ]:
from deeppavlov.utils.telegram import interact_model_by_telegram

interact_model_by_telegram(model_config=gobot_config, token='1153548935:AAFIZkbBaYKjzlpum6wVM6oTHviL4VYlPY8')

2020-07-30 10:10:33.450 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/assistant_bot/word.dict]
2020-07-30 10:10:33.454 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/assistant_bot/glove.6B.100d.txt`]
2020-07-30 10:11:20.54 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-07-30 10:11:20.55 INFO in 'deeppavlov.utils.connector.bot'['bot'] at line 72: Bot initiated
